## Read data

In [1]:
house_1_df = spark.read.format('csv').option('header', True).option('delimiter', ',')\
                .load('housedata/Branch1_HouseDataset.csv')

house_2_df = spark.read.format('csv').option('header', True).option('delimiter', ',')\
                .load('housedata/Branch2_HouseDataset.csv')

gov_1_df = spark.read.format('csv').option('header', True).option('delimiter', ',')\
                .load('housedata/Branch1_GovDataset.csv')

gov_2_df = spark.read.format('csv').option('header', True).option('delimiter', ',')\
                .load('housedata/Branch2_GovDataset.csv')

## Preprocess

#### Change data type

In [2]:
house_1_df = house_1_df.withColumn('Id', house_1_df['Id'].cast('int'))\
                        .withColumn('TotalBsmtSF', house_1_df['TotalBsmtSF'].cast('int'))\
                        .withColumn('1stFlrSF', house_1_df['1stFlrSF'].cast('int'))\
                        .withColumn('2ndFlrSF', house_1_df['2ndFlrSF'].cast('int'))\

In [3]:
house_2_df = house_2_df.withColumn('Id', house_2_df['Id'].cast('int'))\
                        .withColumn('TotalBsmtSF', house_2_df['TotalBsmtSF'].cast('int'))\
                        .withColumn('1stFlrSF', house_2_df['1stFlrSF'].cast('int'))\
                        .withColumn('2ndFlrSF', house_2_df['2ndFlrSF'].cast('int'))\

In [4]:
gov_1_df = gov_1_df.withColumn('Id', gov_1_df['Id'].cast('int'))\
                    .withColumn('LotFrontage', gov_1_df['LotFrontage'].cast('int'))\
                    .withColumn('SalePrice', gov_1_df['SalePrice'].cast('int'))

In [5]:
gov_2_df = gov_2_df.withColumn('Id', gov_2_df['Id'].cast('int'))\
                    .withColumn('LotFrontage', gov_2_df['LotFrontage'].cast('int'))\
                    .withColumn('SalePrice', gov_2_df['SalePrice'].cast('int'))

#### Join and union

In [6]:
house_data_df = gov_1_df.join(house_1_df, 'Id', 'left').unionAll(gov_2_df.join(house_2_df, 'Id', 'left'))
print('Total rows:', house_data_df.count())

Total rows: 1460


In [7]:
house_data_df.show(10)

+---+-----------+--------+---------+-------------+---------+-----+-----------+--------+--------+
| Id|LotFrontage|MSZoning|YearBuilt|SaleCondition|SalePrice|Label|TotalBsmtSF|1stFlrSF|2ndFlrSF|
+---+-----------+--------+---------+-------------+---------+-----+-----------+--------+--------+
|  1|         65|      RL|     2003|       Normal|   208500|    Y|        856|     856|     854|
|  2|         80|      RL|     1976|       Normal|   181500|    Y|       1262|    1262|       0|
|  3|         68|      RL|     2001|       Normal|   223500|    Y|        920|     920|     866|
|  4|         60|      RL|     1915|      Abnorml|   140000|    Y|        756|     961|     756|
|  5|         84|      RL|     2000|       Normal|   250000|    Y|       1145|    1145|    1053|
|  6|         85|      RL|     1993|       Normal|   143000|    Y|        796|     796|     566|
|  7|         75|      RL|     2004|       Normal|   307000|    Y|       1686|    1694|       0|
|  8|       null|      RL|    

## Question: how many house that has only 1st floor?

In [8]:
only_1st_flr_count = house_data_df.filter(
    (house_data_df['TotalBsmtSF'] == 0) 
    & (house_data_df['2ndFlrSF'] == 0) 
    & (house_data_df['1stFlrSF'] != 0)
).count()

print('There are "' + str(only_1st_flr_count) + '" houses that have only 1st floor.')

There are "27" houses that have only 1st floor.


## Question: Calculate price per square feet of total area

In [9]:
house_data_df = house_data_df.withColumn('TotalArea', house_data_df['TotalBsmtSF'] + house_data_df['1stFlrSF'] + house_data_df['2ndFlrSF'])
house_data_df = house_data_df.withColumn('PricePerSF', house_data_df['SalePrice'] / house_data_df['TotalArea'])

house_data_df.show(10)

+---+-----------+--------+---------+-------------+---------+-----+-----------+--------+--------+---------+-----------------+
| Id|LotFrontage|MSZoning|YearBuilt|SaleCondition|SalePrice|Label|TotalBsmtSF|1stFlrSF|2ndFlrSF|TotalArea|       PricePerSF|
+---+-----------+--------+---------+-------------+---------+-----+-----------+--------+--------+---------+-----------------+
|  1|         65|      RL|     2003|       Normal|   208500|    Y|        856|     856|     854|     2566|81.25487139516757|
|  2|         80|      RL|     1976|       Normal|   181500|    Y|       1262|    1262|       0|     2524|71.90966719492869|
|  3|         68|      RL|     2001|       Normal|   223500|    Y|        920|     920|     866|     2706|82.59423503325942|
|  4|         60|      RL|     1915|      Abnorml|   140000|    Y|        756|     961|     756|     2473|56.61140315406389|
|  5|         84|      RL|     2000|       Normal|   250000|    Y|       1145|    1145|    1053|     3343|74.78312892611427|
